## Coaxial Line Properties

Electrical properties of a coaxial transmission line are calculated in the `coax` function. It returns multiple values in a namedtuple so suncal can detect the names of all the output parameters and provide uncertainties simultaneously.

To use units (via the Pint Python package), suncal's unit registry must be used (only a single UnitRegistry instance is allowed per application). The UnitRegistry is imported from suncal with `from suncal import ureg`. In this example, units are needed for the constants such as $\epsilon_0$ and $\mu_0$ in the `coax` function.

In [1]:
import numpy as np
from collections import namedtuple

import suncal
from suncal import ureg

In [2]:
mu0 = 1.25663706212E-6 * ureg.henry / ureg.meter  # H/m
eps0 = 8.8541878128E-12 * ureg.farad / ureg.meter # F/m
c = 299792458 * ureg.meter/ureg.second            # m/s

def coax(a, b, freq, rho, mu_r, eps_r, TL):
    ''' Calculate properties of coaxial line '''
    omega = 2 * np.pi * freq
    L = mu0 * mu_r / 2 / np.pi * np.log(b/a)   # H/m
    C = 2 * np.pi * eps0 * eps_r / np.log(b/a) # F/m
    R = np.sqrt(rho * freq * mu0 * mu_r / np.pi) * (1/a + 1/b) / 2   # ohm/m
    G = TL * np.pi * 2 * freq * C   # S/m
    Rdc = rho / np.pi / a**2        # DC resistance, ohm/m
    propvel = 1/np.sqrt(L*C)/c      # propagation velocity as fraction of c
    Z0 = abs(np.sqrt((R + 1j*omega*L)/(G + 1j*omega*C)))      # ohm
    atten = np.sqrt((R + 1j*omega*L)*(G + 1j*omega*C)).real   # Np/m

    coaxresult = namedtuple('Coax', ['L', 'C', 'R', 'G', 'Rdc', 'velocity', 'Z0', 'gamma'])
    return coaxresult(L, C, R, G, Rdc, propvel, Z0, atten)

In [3]:
uc = suncal.ModelCallable(coax)
uc.var('a').measure(0.089/2, units='inch').typeb(unc='1%', units='inch')            # inner radius
uc.var('b').measure(0.285/2, units='inch').typeb(unc='1%', units='inch')            # outer radius
uc.var('freq').measure(100, units='MHz')                        # frequency
uc.var('rho').measure(2.21E-8, units='ohm*m').typeb(unc='1%', units='ohm*m')         # effective resistivity of center conductor (copper)
uc.var('mu_r').measure(1, units='dimensionless')                # permeability
uc.var('eps_r').measure(2.115, units='dimensionless').typeb(unc='1%', units='dimensionless') # permittivity of PE that results in 30.8 pF/ft
uc.var('TL').measure(.0002, units='dimensionless').typeb(unc='1%', units='dimensionless')    # Loss tangent of PE (depends on frequency - this value is for 100 MHz)
result = uc.calculate_gum().units(L ='uH/m', C='pF/ft', R='ohm/m', G='1/ohm/m', Rdc='ohm/hectofeet',
                             velocity='dimensionless', Z0='ohm', gamma='1/hectofoot')
result

|Function   | Nominal   | Std. Uncertainty  |
|----------|---------|------------------|
|$L$   | 0.2328 µH/m  | 0.0028 µH/m          |
|$C$   | 30.81 pF/ft  | 0.48 pF/ft          |
|$R$   | 0.5458 Ω/m  | 0.0051 Ω/m          |
|$G$   | 0.00001270 1/m/Ω  | 2.4e-07 1/m/Ω          |
|$Rdc$   | 0.1678 Ω/hft  | 0.0038 Ω/hft          |
|$velocity$   | 0.6876  | 0.0034          |
|$Z_{0}$   | 47.98 Ω | 0.63 Ω         |
|$\gamma$   | 0.1826 1/hft | 0.0023 1/hft         |

In [4]:
result.report.sensitivity()

Variable,Sensitivity,Proportion
$TL$,0.0 H/m,0.00%
$a$,-4.5e-06 H/in/m,50.00%
$b$,1.4e-06 H/in/m,50.00%
$eps_{r}$,0.0 H/m,0.00%
$freq$,0.0 H/MHz/m,0.00%
$\mu_{r}$,2.3e-07 H/m,0.00%
$\rho$,0.0 H/m²/Ω,0.00%
Variable,Sensitivity,Proportion
$TL$,0.0 F/m,0.00%
$a$,2.0e-09 F/in/m,29.81%
